In [2]:
import torch
import numpy as np
import scipy.io as sio

### Import Data 

In [3]:
data = sio.loadmat('data/cylinder_nektar_wake.mat')

In [9]:
U_star = data['U_star'] # N x 2 x T
p_star = data['p_star'] # N x 2 x T
t_star = data ['t'] # T x 1
X_star = data['X_star'] # N x 2 

N = X_star.shape[0] # 5000
T = t_star.shape[0] # 200

### Format data

In [10]:
x_test = X_star[:, 0:1]
y_test = X_star[:, 1:2]
p_test = p_star[:, 0:1]
u_test = U_star[:, 0:1, 0]
t_test = np.ones((x_test.shape[0], x_test.shape[1]))

In [11]:
XX = np.tile(X_star[:, 0:1], (1, T))  # N x T
YY = np.tile(X_star[:, 1:2], (1, T))  # N x T
TT = np.tile(t_star, (1, N)).T  # N x T

UU = U_star[:, 0, :]  # N x T
VV = U_star[:, 1, :]  # N x T
PP = p_star  # N x T

x = XX.flatten()[:, None]  # NT x 1
y = YY.flatten()[:, None]  # NT x 1
t = TT.flatten()[:, None]  # NT x 1

u = UU.flatten()[:, None]  # NT x 1
v = VV.flatten()[:, None]  # NT x 1
p = PP.flatten()[:, None]  # NT x 1

### Select Train and Test data and add noise 

In [8]:
np.random.seed(42)
torch.manual_seed(42)

In [15]:
# Noiseless data
N_train = 5000
idx = np.random.choice(N*T, N_train, replace=False)
x_train = x[idx,:] # 5000 x 1
y_train = y[idx,:]
t_train = t[idx,:]
u_train = u[idx,:]
v_train = v[idx,:]

In [16]:
# Noisy data
noise = 0.01        
u_train = u_train + noise*np.std(u_train)*np.random.randn(u_train.shape[0], u_train.shape[1])
v_train = v_train + noise*np.std(v_train)*np.random.randn(v_train.shape[0], v_train.shape[1])

### Model

In [37]:
class PINN():


    def __init__(self, x, y, t, u, v):
        
        self.x = torch.tensor(x, dtype=torch.float32, requires_grad=True)
        self.y = torch.tensor(y, dtype=torch.float32, requires_grad=True)
        self.t = torch.tensor(t, dtype=torch.float32, requires_grad=True)
        self.u = torch.tensor(u, dtype=torch.float32, requires_grad=True)
        self.v = torch.tensor(v, dtype=torch.float32, requires_grad=True)


        self.null = torch.zeros_like(self.x)

        self.neuralnet()

        self.optimizer = torch.optim.LBFGS(self.net.parameters(), lr= 1, max_iter= 10000, max_eval= 5000, history_size= 50, line_search_fn= 'strong_wolfe', tolerance_change= 0.5* np.finfo(float).eps, tolerance_grad= 1e-05)

        self.mse = torch.nn.MSELoss()
        self.loss = 0 
        self.iter = 0


    def neuralnet(self):

        self.net = torch.nn.Sequential(
            torch.nn.Linear(3, 20), torch.nn.Tanh(),
            torch.nn.Linear(20, 20), torch.nn.Tanh(),
            torch.nn.Linear(20, 20), torch.nn.Tanh(),
            torch.nn.Linear(20, 20), torch.nn.Tanh(),
            torch.nn.Linear(20, 20), torch.nn.Tanh(),
            torch.nn.Linear(20, 20), torch.nn.Tanh(),
            torch.nn.Linear(20, 20), torch.nn.Tanh(),
            torch.nn.Linear(20, 20), torch.nn.Tanh(),
            torch.nn.Linear(20, 20), torch.nn.Tanh(),
            torch.nn.Linear(20, 2))
        
    def navier_stokes(self, x, y, t):
        lambda_1, lambda_2 = 1., 1.

        result = self.net(torch.cat([x, y, t], 1))
        psi, p = result[:, 0:1], result[:, 1:2]

        u = torch.autograd.grad(psi, y, torch.ones_like(x), create_graph=True)[0]
        v = -torch.autograd.grad(psi, x, torch.ones_like(y), create_graph=True)[0]

        p_x = torch.autograd.grad(p, x, torch.ones_like(x), create_graph=True)[0]
        p_y = torch.autograd.grad(p, y, torch.ones_like(y), create_graph=True)[0]

        u_t = torch.autograd.grad(u, t, torch.ones_like(t), create_graph=True)[0]
        u_x = torch.autograd.grad(u, x, torch.ones_like(x), create_graph=True)[0]
        u_y = torch.autograd.grad(u, y, torch.ones_like(y), create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x, torch.ones_like(x), create_graph=True)[0]
        u_yy = torch.autograd.grad(u_y, y, torch.ones_like(y), create_graph=True)[0]

        v_t = torch.autograd.grad(v, t, torch.ones_like(t), create_graph=True)[0]
        v_x = torch.autograd.grad(v, x, torch.ones_like(x), create_graph=True)[0]
        v_y = torch.autograd.grad(v, y, torch.ones_like(y), create_graph=True)[0]
        v_xx = torch.autograd.grad(v_x, x, torch.ones_like(x), create_graph=True)[0]
        v_yy = torch.autograd.grad(v_y, y, torch.ones_like(y), create_graph=True)[0]

        f_u = u_t + lambda_1 * (u*u_x + v*u_y) + p_x - lambda_2 * (u_xx + u_yy)
        f_v = v_t + lambda_1 * (u*v_x + v*v_y) + p_y - lambda_2 * (v_xx + v_yy)

        return u, v, f_u, f_v
    
    def closure(self):

        self.optimizer.zero_grad()

        u_pred, v_pred, f_u_pred, f_v_pred = self.navier_stokes(self.x, self.y, self.t)

        loss_u = self.mse(u_pred, self.u)
        loss_v = self.mse(v_pred, self.v)
        loss_f_u = self.mse(f_u_pred, self.null)
        loss_f_v = self.mse(f_v_pred, self.null)

        self.loss = loss_u + loss_v + loss_f_u + loss_f_v

        self.loss.backward()

        self.iter += 1
        if not self.iter % 10:
            print("Iteration: {:} Loss: {:.6e}".format(self.iter, self.loss))

        return self.loss

    def train_nn(self):
        
        self.net.train()
        self.optimizer.step(self.closure)

In [39]:
N_train = 5000

pinn_navier_stokes = PINN(x_train, y_train, t_train, u_train, v_train)
pinn_navier_stokes.train_nn()

Iteration: 10 Loss: 6.637693e+14
Iteration: 20 Loss: 1.219811e+00
Iteration: 30 Loss: 2.572105e-01
Iteration: 40 Loss: 1.959958e-01
Iteration: 50 Loss: 1.732053e-01
Iteration: 60 Loss: 1.619789e-01
Iteration: 70 Loss: 1.558117e-01
Iteration: 80 Loss: 1.497016e-01
Iteration: 90 Loss: 1.462971e-01
Iteration: 100 Loss: 1.363193e-01
Iteration: 110 Loss: 1.337016e-01
Iteration: 120 Loss: 1.319977e-01
Iteration: 130 Loss: 1.307526e-01
Iteration: 140 Loss: 1.296176e-01
Iteration: 150 Loss: 1.288105e-01
Iteration: 160 Loss: 1.281347e-01
Iteration: 170 Loss: 1.277159e-01
Iteration: 180 Loss: 1.271963e-01
Iteration: 190 Loss: 1.266118e-01
Iteration: 200 Loss: 1.260761e-01
Iteration: 210 Loss: 1.259084e-01
Iteration: 220 Loss: 1.254972e-01
Iteration: 230 Loss: 1.252136e-01
Iteration: 240 Loss: 1.250387e-01
Iteration: 250 Loss: 1.248327e-01
Iteration: 260 Loss: 1.245429e-01
Iteration: 270 Loss: 1.243961e-01
Iteration: 280 Loss: 1.242992e-01
Iteration: 290 Loss: 1.241465e-01
Iteration: 300 Loss: 1.

In [41]:
torch.save(pinn_navier_stokes.net, 'pinn_navier_stokes.pth')